<a href="https://colab.research.google.com/github/rasiq-gulzar/data_analytics/blob/main/new_pandas_melbourne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [149]:
import pandas as pd
import re

In [150]:
df=pd.read_csv('/content/melb_data.csv')

In [151]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [152]:
df.shape

(13580, 21)

In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [154]:
df=df.fillna(0)

In [155]:
# Map Type codes to full names
type_mapping = {'h': 'House', 'u': 'Unit', 't': 'Townhouse'}
df['Type'] = df['Type'].map(type_mapping)

# Map Method codes (common real estate terms)
method_mapping = {
    'S': 'Sold', 'SP': 'Sold Prior', 'PI': 'Passed In',
    'VB': 'Vendor Bid', 'SA': 'Sold After'
}
df['Method'] = df['Method'].map(method_mapping)

In [156]:
# Check for unrealistic values
print("Price outliers:", df['Price'].describe())
print("Landsize outliers:", df['Landsize'].describe())
print("BuildingArea outliers:", df['BuildingArea'].describe())

# Remove extreme outliers (adjust thresholds as needed)
df = df[df['Price'] > 0]  # Remove zero prices
df = df[df['YearBuilt'] > 1800]  # Remove unrealistic build years

Price outliers: count    1.358000e+04
mean     1.075684e+06
std      6.393107e+05
min      8.500000e+04
25%      6.500000e+05
50%      9.030000e+05
75%      1.330000e+06
max      9.000000e+06
Name: Price, dtype: float64
Landsize outliers: count     13580.000000
mean        558.416127
std        3990.669241
min           0.000000
25%         177.000000
50%         440.000000
75%         651.000000
max      433014.000000
Name: Landsize, dtype: float64
BuildingArea outliers: count    13580.000000
mean        79.788611
std        399.281619
min          0.000000
25%          0.000000
50%         51.000000
75%        129.940000
max      44515.000000
Name: BuildingArea, dtype: float64


In [157]:
# Standardize suburb names
df['Suburb'] = df['Suburb'].str.title().str.strip()

# Clean address formatting
df['Address'] = df['Address'].str.strip()

In [158]:
# Fix the typo
df.rename(columns={'Lattitude': 'Latitude'}, inplace=True)

In [159]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Latitude,Longtitude,Regionname,Propertycount
1,Abbotsford,25 Bloomburg St,2,House,1035000.0,Sold,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,House,1465000.0,Sold Prior,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,House,1600000.0,Vendor Bid,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
6,Abbotsford,124 Yarra St,3,House,1876000.0,Sold,Nelson,7/05/2016,2.5,3067.0,...,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
7,Abbotsford,98 Charles St,2,House,1636000.0,Sold,Nelson,8/10/2016,2.5,3067.0,...,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0


In [160]:
# Convert to appropriate data types
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Postcode'] = df['Postcode'].astype(str)
df['Rooms'] = df['Rooms'].astype(int)
df['Bathroom'] = df['Bathroom'].astype(int)
df['Car'] = df['Car'].astype(int)
df['YearBuilt'] = df['YearBuilt'].astype(int)

In [161]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Latitude,Longtitude,Regionname,Propertycount
1,Abbotsford,25 Bloomburg St,2,House,1035000.0,Sold,Biggin,2016-02-04,2.5,3067.0,...,1,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,House,1465000.0,Sold Prior,Biggin,2017-03-04,2.5,3067.0,...,2,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,House,1600000.0,Vendor Bid,Nelson,2016-06-04,2.5,3067.0,...,1,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
6,Abbotsford,124 Yarra St,3,House,1876000.0,Sold,Nelson,2016-05-07,2.5,3067.0,...,2,0,245.0,210.0,1910,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
7,Abbotsford,98 Charles St,2,House,1636000.0,Sold,Nelson,2016-10-08,2.5,3067.0,...,1,2,256.0,107.0,1890,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0


In [163]:
# Extract number + letter as street number
df['Street Number'] = df['Address'].str.extract(r'^(\d+[A-Za-z]*)')

# Remove number + letter + space from address
df['Address'] = df['Address'].str.replace(r'^\d+[A-Za-z]*\s+', '', regex=True)

In [164]:
df

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Latitude,Longtitude,Regionname,Propertycount,Street Number
1,Abbotsford,Bloomburg St,2,House,1035000.0,Sold,Biggin,2016-02-04,2.5,3067.0,...,0,156.0,79.0,1900,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0,25
2,Abbotsford,Charles St,3,House,1465000.0,Sold Prior,Biggin,2017-03-04,2.5,3067.0,...,0,134.0,150.0,1900,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0,5
4,Abbotsford,Park St,4,House,1600000.0,Vendor Bid,Nelson,2016-06-04,2.5,3067.0,...,2,120.0,142.0,2014,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0,55a
6,Abbotsford,Yarra St,3,House,1876000.0,Sold,Nelson,2016-05-07,2.5,3067.0,...,0,245.0,210.0,1910,Yarra,-37.80240,144.99930,Northern Metropolitan,4019.0,124
7,Abbotsford,Charles St,2,House,1636000.0,Sold,Nelson,2016-10-08,2.5,3067.0,...,2,256.0,107.0,1890,Yarra,-37.80600,144.99540,Northern Metropolitan,4019.0,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,Strada Cr,4,House,1245000.0,Sold,Barry,2017-08-26,16.7,3150.0,...,2,652.0,0.0,1981,0,-37.90562,145.16761,South-Eastern Metropolitan,7392.0,12
13576,Williamstown,Merrett Dr,3,House,1031000.0,Sold Prior,Williams,2017-08-26,6.8,3016.0,...,2,333.0,133.0,1995,0,-37.85927,144.87904,Western Metropolitan,6380.0,77
13577,Williamstown,Power St,3,House,1170000.0,Sold,Raine,2017-08-26,6.8,3016.0,...,4,436.0,0.0,1997,0,-37.85274,144.88738,Western Metropolitan,6380.0,83
13578,Williamstown,Verdon St,4,House,2500000.0,Passed In,Sweeney,2017-08-26,6.8,3016.0,...,5,866.0,157.0,1920,0,-37.85908,144.89299,Western Metropolitan,6380.0,96
